In [12]:
#!/usr/bin/env python
# coding: utf-8

"""
Simple client code for the ESP32 in Python - adapted from OFM Client 
Copyright 2020 Richard Bowman, released under LGPL 3.0 or later
Copyright 2021 Benedict Diederich, released under LGPL 3.0 or later
"""

import requests
import json
import time
import io
#import PIL.Image
import numpy as np
import logging
import zeroconf
import requests 
import json
import time 
#import matplotlib.pyplot as plt

ACTION_RUNNING_KEYWORDS = ["idle", "pending", "running"]
ACTION_OUTPUT_KEYS = ["output", "return"]

class ESP32Client(object):
    headers = {'ESP32-version': '*'}

    def __init__(self, host, port=31950):
        if isinstance(host, zeroconf.ServiceInfo):
            # If we have an mDNS ServiceInfo object, try each address
            # in turn, to see if it works (sometimes you get addresses
            # that don't work, if your network config is odd).
            # TODO: figure out why we can get mDNS packets even when
            # the microscope is unreachable by that IP
            for addr in host.parsed_addresses():
                if ":" in addr:
                    self.host = f"[{addr}]"
                else:
                    self.host = addr
                self.port = host.port
                try:
                    self.get_json(self.base_uri)
                    break
                except:
                    logging.info(f"Couldn't connect to {addr}, we'll try another address if possible.")
        else:
            self.host = host
            self.port = port
            #self.get_json(self.base_uri)
        logging.info(f"Connecting to microscope {self.host}:{self.port}")
        #self.populate_extensions()

    extensions = None
        
    @property
    def base_uri(self):
        return f"http://{self.host}:{self.port}"

    def get_json(self, path):
        """Perform an HTTP GET request and return the JSON response"""
        if not path.startswith("http"):
            path = self.base_uri + path
        r = requests.get(path)
        r.raise_for_status()
        return r.json()

    def post_json(self, path, payload={}):
        """Make an HTTP POST request and return the JSON response"""
        if not path.startswith("http"):
            path = self.base_uri + path
        r = requests.post(path, json=payload, headers=self.headers)
        r.raise_for_status()
        r = r.json()
        return r


    def get_temperature(self):
        path = "/temperature"
        r = self.get_json(path)
        return r['value']
    
    #%% LED
    def set_led(self, colour=(0,0,0)):
        payload = {
            "red": colour[0], 
            "green": colour[1], 
            "blue": colour[2]
        }
        path = '/led'
        r = self.post_json(path, payload)
        return r
    
    def move_x(self, steps=100, speed=10):
        payload = {
            "steps": steps, 
            "speed": speed,            
        }
        path = '/move_x'
        r = self.post_json(path, payload)
        return r
    
    def move_y(self, steps=100, speed=10):
        payload = {
            "steps": steps, 
            "speed": speed,            
        }
        path = '/move_y'
        r = self.post_json(path, payload)
        return r
    
    def lens_x(self, value=100):
        payload = {
            "lens_value": value,            
        }
        path = '/lens_x'
        r = self.post_json(path, payload)
        return r
    
    def lens_z(self, value=100):
        payload = {
            "lens_value": value,            
        }
        path = '/lens_z'
        r = self.post_json(path, payload)
        return r
    

host = '192.168.43.33'
esp32 = ESP32Client(host, port=80)

    

In [13]:
import numpy as np
import matplotlib.pyplot as plt
from laser import laser
from lens import lens

import time
from IPython.display import clear_output
import matplotlib
from IPython.display import display 
import ipywidgets as widgets 
from ipywidgets import interact, Layout 


In [14]:
slider1 = widgets.IntSlider(value=1000,min=0,max=10000,step=100,description='LensX:',orientation='vertical')

In [17]:
def updatelens1(lensvalX): 
    esp32.lens_x(value=lensvalX) 

def updatelens2(lensvalZ): 
    esp32.lens_z(value=lensvalZ)
    
interact(updatelens1, lensvalX = (0,25000))
interact(updatelens2, lensvalZ = (0,25000))

interactive(children=(IntSlider(value=12500, description='lensvalX', max=25000), Output()), _dom_classes=('wid…

interactive(children=(IntSlider(value=12500, description='lensvalZ', max=25000), Output()), _dom_classes=('wid…

<function __main__.updatelens2(lensvalZ)>

In [8]:
esp32.move_y(steps=5000, speed=6)

{}

In [ ]:

    

esp32.set_led((100,2,5))
#esp32.move_x(steps=2000, speed=8)
#esp32.move_y(steps=2000, speed=6)

#%%
esp32.lens_x(value=10000)
esp32.lens_z(value=10000)
#%%
for ix in range(0,32000,100):
    esp32.lens_x(value=ix)
    for iy in range(0,32000,100):
        esp32.lens_z(value=iy)
esp32.lens_z(value=0)
esp32.lens_x(value=0)

#%%
esp32.lens_x(value=8000)
esp32.lens_x(value=20900)

#%%
for iy in range(20000,23000,100):
    time.sleep(.05)
    print(iy)
    esp32.lens_z(value=iy)
    
#%%
esp32.move_y(steps=-10000, speed=6)

    